In [ ]:
!pip install scenedetect[opencv]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 6.4 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import subprocess
import cv2
from scenedetect.detectors import AdaptiveDetector

def detect_scenes_dynamic(input_path,
                          motion_threshold: float = 5.0,
                          min_skip: int = 0,
                          max_skip: int = 5,
                          adaptive_threshold: float = 3.0,
                          window_width: int = 2,
                          min_content_val: float = 15.0):
    """
    모션 변화량에 따라 프레임 스킵을 조정하며 AdaptiveDetector로 씬(컷) 구간을 계산합니다.

    Args:
        input_path (str): 입력 비디오 경로
        motion_threshold (float): 모션 값(motion_val)이 이 이상이면 skip=min_skip
        min_skip (int): 모션 클 때 최소 스킵 프레임 수
        max_skip (int): 모션 작을 때 최대 스킵 프레임 수
        adaptive_threshold (float): AdaptiveDetector adaptive ratio 기준 (높을수록 덜 민감)
        window_width (int): adaptive rolling average 윈도우 크기
        min_content_val (float): 컷으로 인식되려면 content score 최소값

    Returns:
        List[Tuple[float, float]]: (start_sec, end_sec) 씬 시간 리스트
    """
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {input_path}")
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    detector = AdaptiveDetector(
        adaptive_threshold=adaptive_threshold,
        window_width=window_width,
        min_content_val=min_content_val,
        min_scene_len=1
    )

    ret, prev_frame = cap.read()
    if not ret:
        cap.release()
        return []
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    cut_frame_nums = []
    frame_num = 1

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        motion_val = cv2.absdiff(prev_gray, gray).mean()
        skip = min_skip if motion_val > motion_threshold else max_skip

        cuts = detector.process_frame(frame_num, frame)
        if cuts:
            cut_frame_nums.extend(cuts)

        for _ in range(skip):
            if not cap.grab():
                break
            frame_num += 1

        prev_gray = gray
        frame_num += 1

    cuts = detector.post_process(frame_num)
    cut_frame_nums.extend(cuts)
    cap.release()

    cut_frame_nums = sorted(set(cut_frame_nums))
    boundaries = [0] + cut_frame_nums + [total_frames]
    scenes = [(boundaries[i]/fps, boundaries[i+1]/fps)
              for i in range(len(boundaries)-1)]
    return scenes


def merge_short_scenes(scenes, min_duration):
    """
    scenes 중 duration이 min_duration 미만인 씬은 이전 씬과 병합합니다.

    Args:
        scenes (List[Tuple[float, float]]): 원본 씬 리스트
        min_duration (float): 최소 씬 길이(초)

    Returns:
        List[Tuple[float, float]]: 병합 후 씬 리스트
    """
    if not scenes:
        return []
    merged = [scenes[0]]
    for start, end in scenes[1:]:
        prev_start, prev_end = merged[-1]
        duration = end - start
        if duration < min_duration:
            # 이전 씬의 끝을 현재 씬의 끝으로 확장
            merged[-1] = (prev_start, end)
        else:
            merged.append((start, end))
    return merged


def split_scenes(input_path, scenes, output_dir="scenes"):
    """
    FFmpeg를 이용해 씬 구간별로 영상을 분할 저장합니다.
    이전 output_dir이 존재하면 삭제 후 새로 생성합니다.
    """
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    for idx, (start, end) in enumerate(scenes, 1):
        output_file = os.path.join(output_dir, f"Scene_{idx}.mp4")
        cmd = [
            'ffmpeg', '-y', '-i', input_path,
            '-ss', f"{start:.3f}", '-to', f"{end:.3f}",
            '-c', 'copy', output_file
        ]
        print(f"Splitting Scene {idx}: {start:.2f}s → {end:.2f}s")
        subprocess.run(cmd, check=True)

    print("All scenes have been split and saved to:", output_dir)


if __name__ == "__main__":
    video_file = "/content/test_video.mp4"

    scenes = detect_scenes_dynamic(
        video_file,
        motion_threshold=3.0,
        min_skip=0,
        max_skip=5,
        adaptive_threshold=3.2,
        window_width=4,
        min_content_val=20.0
    )
    # 영상의 길이에 따라 n초 미만 씬은 병합
    scenes = merge_short_scenes(scenes, min_duration=5.0)
    split_scenes(video_file, scenes, output_dir="output_scenes")


Splitting Scene 1: 0.00s → 30.60s
Splitting Scene 2: 30.60s → 39.47s
Splitting Scene 3: 39.47s → 49.43s
Splitting Scene 4: 49.43s → 59.77s
All scenes have been split and saved to: output_scenes


In [ ]:
!zip -r output_scenes.zip output_scenes

  adding: output_scenes/ (stored 0%)
  adding: output_scenes/Scene_5.mp4 (deflated 1%)
  adding: output_scenes/Scene_6.mp4 (deflated 1%)
  adding: output_scenes/Scene_3.mp4 (deflated 1%)
  adding: output_scenes/Scene_2.mp4 (deflated 6%)
  adding: output_scenes/Scene_7.mp4 (deflated 0%)
  adding: output_scenes/Scene_4.mp4 (deflated 3%)
  adding: output_scenes/Scene_1.mp4 (deflated 3%)
